Install TensorFlow Version 1

In [ ]:
!pip install 'tensorflow==1.15.3'
!pip install 'tensorflow-gpu==1.15.3'
!pip install 'keras==2.2.4'

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Import Required Packages

In [ ]:
import os
from os import listdir

import sys
import json
import skimage.io
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("/content/drive/MyDrive/Project/Mask_RCNN-master/Mask_RCNN-master/")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

from mrcnn import config
from mrcnn.config import Config
from mrcnn.utils import Dataset
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log


%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")


Save some Variables that will be used later

In [ ]:
Num_Classes = 5
#Weight_File = '/content/drive/MyDrive/Project/Mask_RCNN-master/Mask_RCNN-master/logs/final_model20201209T2024/mask_rcnn_final_model_0015.h5'
Train_Annot_File = "/content/drive/MyDrive/Project/Sets/NewTraining/new_annotation.json"
Train_Annot_Image_Dir = "/content/drive/MyDrive/Project/Sets/NewTraining"
Val_Annot_File = "/content/drive/MyDrive/Project/Sets/Training/new_annotation.json"
Val_Annot_Image_Dir = "/content/drive/MyDrive/Project/Sets/Training"
Model_Name = "Final_Model"

Override Config for Custom Training

In [ ]:
class TrainConfig(Config):
    # define the name of the configuration
    NAME = Model_Name
    # number of classes (background + others)
    NUM_CLASSES = 1 + 5
    # number of training steps per epoch
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    STEPS_PER_EPOCH=2000
    
    
    ###################################################################################
    BACKBONE = "resnet50"
    IMAGES_PER_GPU = 1
    
    # Length of square anchor side in pixels
    RPN_ANCHOR_SCALES = (16, 32, 64, 128, 256)
    
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.6
    
    # How many anchors per image to use for RPN training
    RPN_TRAIN_ANCHORS_PER_IMAGE = 512
    
    # Percent of positive ROIs used to train classifier/mask heads
    ROI_POSITIVE_RATIO = 0.6
    
#     IMAGE_MIN_DIM = 512
#     IMAGE_MAX_DIM = 1024
    
    # Number of ROIs per image to feed to classifier/mask heads
    # The Mask RCNN paper uses 512 but often the RPN doesn't generate
    # enough positive proposals to fill this and keep a positive:negative
    # ratio of 1:3. You can increase the number of proposals by adjusting
    # the RPN NMS threshold.
    #TRAIN_ROIS_PER_IMAGE = 512
    
    # Max number of final detections
    DETECTION_MAX_INSTANCES = 100
    
    # Minimum probability value to accept a detected instance
    # ROIs below this threshold are skipped
    DETECTION_MIN_CONFIDENCE = 0.5

    # Non-maximum suppression threshold for detection
    DETECTION_NMS_THRESHOLD = 0.3
    pass
 
# prepare config
config = TrainConfig()
config.display()



Create Functions to Load the Dataset and Load the Masks. This code was given by the balloon.py implementation of Mask RCNN and was adapted for our Custom Dataset.

In [ ]:

class Train_Dataset(utils.Dataset):

    def load_dataset(self, dataset_dir):
        """Load the dataset.
        """
        # Add classes
        self.add_class("dataset", 1, "Head")
        self.add_class("dataset", 2, "Thread")
        self.add_class("dataset", 3, "Nut")
        self.add_class("dataset", 4, "Washer")
        self.add_class("dataset", 5, "Pin")

        # Load annotations
        # VGG Image Annotator (up to version 1.6) saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        # Note: In VIA 2.0, regions was changed from a dict to a list.
        annotations = json.load(open(os.path.join(dataset_dir, "new_annotation.json")))
        annotations = list(annotations.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
      
        # Add images
        for a in annotations:
            
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. These are stores in the
            # shape_attributes 
            
            polygons = [r['shape_attributes'] for r in a['regions']] 
            category = [c['region_attributes']['class'] for c in a['regions']]
            classes = {"Head":1,"Thread":2,"Nut":3,"Washer":4,"Pin":5}    
            num_ids = [classes[a] for a in category]
            
            

            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "dataset",
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids)

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: array of class IDs of the instance masks.
        """
        # If not a dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "dataset":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        
        return mask.astype(np.bool), np.array(num_ids, dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "dataset":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)
    

Load the Training and Validation Datasets

In [ ]:
# Training dataset
dataset_train = Train_Dataset()
dataset_train.load_dataset(Train_Annot_Image_Dir)
dataset_train.prepare()
# Validation dataset
dataset_val = Train_Dataset()
dataset_val.load_dataset(Val_Annot_Image_Dir)
dataset_val.prepare()

Create a Model to Train

In [ ]:
model = modellib.MaskRCNN(mode = 'training', config = TrainConfig(), model_dir = MODEL_DIR)

Train model: 2000 Steps per Epochs with 15 Epochs

In [ ]:
start_train = time.time()
model.train(dataset_train,dataset_val,learning_rate= TrainConfig().LEARNING_RATE, epochs = 15, layers = 'heads')
end_train = time.time()
minutes = round((end_train - start_train) / 60, 2)
print(minutes)

Use this to Manually Save Weights 

In [ ]:
#model_path = os.path.join(MODEL_DIR, "mask_rcnn_weights_v6.h5")
#model.keras_model.save_weights(model_path)